# Make Sure You Are Ready to Go

$\renewcommand{\ket}[1]{\left| #1 \right\rangle}
\renewcommand{\bra}[1]{\left\langle #1 \right|}
\renewcommand{\braket}[2]{\left\langle #1 | #2 \right\rangle}
\newcommand{\ketbra}[2]{\left| #1 \right\rangle\!\left\langle #2 \right|}$

If you haven't done it yet, try running the following lines of code and use the [registration and installation](https://docs.classiq.io/latest/classiq_101/registration_installations/) page if you are having difficulty setting up your environment.\
Uncomment and run the following command to install or update to the latest version of the Classiq SDK (if not installed yet):

In [1]:
pip install -U classiq

In [2]:
import classiq

Uncomment and run the following command if your machine has not been

authenticated yet, you only need to run it once!

In [3]:
pip install keyrings.alt

In [4]:
#from keyrings.alt.file import PlaintextKeyring

# # Use a basic keyring backend that stores credentials insecurely (for environments like Colab or VS Code)
#keyring.set_keyring(PlaintextKeyring())

# import classiq
classiq.authenticate()

Your user code: TBZW-SZNJ
If a browser doesn't automatically open, please visit this URL from any trusted device: https://auth.classiq.io/activate?user_code=TBZW-SZNJ
Please set a password for your new keyring: ··········
Please confirm the password: ··········


Now you are good to go!

# Rydberg Phase Diagram

Before starting to code, let us reiterating some theory on Rydberg atoms - the subject of this challenge. They interact via the following Hamiltonian:

$$
H = \frac{\Omega}{2} \sum_{i=1}^N X_i
    - \delta \sum_{i=1}^N n_i
    + \sum_{i \lt j} \frac{\Omega R_b^6 }{(a|i-j|)^6} n_i n_j.
$$

You can find the phase diagram for a $51$-atom chain below. It is obtained by fixing $a=1$ and $\Omega=1$ and varying $\delta$ and $R_b$.

<img src="https://github.com/dmitriikhitrin/Classiq-x-DuQIS-FLIQ-Challenge/blob/main/phase_diagram.png?raw=1" alt="Phase Diagram" width="800">


Fig.1: Phase diagram of the 1D Rydberg Hamiltonian, traced out by (left) bipartite entanglement entropy and (right) expectation value of the number of Rydberg excitations. Plots are obtained using tensor-network representation of the ground states of $H$.

In this challenge, we focus on distinguishing between the $Z2$ phase, where the ground state of $H$ has large overlap with the state $\ket{rgr\ldots gr}$, and the $Z3$ phase, where the ground state overlaps strongly with basis states of the form $\ket{\ldots rggrgg\ldots}$.

Evidently, such systems can be efficiently studied using tensor networks. However, this challenge prepares us for a more realistic scenario in which we only have access to measurement outcomes from the ground state of some Hamiltonian, and our goal is to determine which phase of matter the state belongs to.

# Loading and Processing Measurement Data

*   List item
*   List item



Training data for your model contains measurement results in randomized bases performed on a 51-qubit Rydberg atoms chain. We load training data from the .npz file in the next cell.

In [5]:
!wget https://github.com/dmitriikhitrin/Classiq-x-DuQIS-FLIQ-Challenge/raw/main/training_data.npz


--2025-05-17 20:28:22--  https://github.com/dmitriikhitrin/Classiq-x-DuQIS-FLIQ-Challenge/raw/main/training_data.npz
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dmitriikhitrin/Classiq-x-DuQIS-FLIQ-Challenge/main/training_data.npz [following]
--2025-05-17 20:28:22--  https://raw.githubusercontent.com/dmitriikhitrin/Classiq-x-DuQIS-FLIQ-Challenge/main/training_data.npz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1021888 (998K) [application/octet-stream]
Saving to: ‘training_data.npz’

training_data.npz   100%[===================>] 997.94K  --.-KB/s    in 0.04s   

2025-05-17 20:28:23 (23.6

In [6]:
import numpy as np
# You might need to make additional imports depending on your implementation

In [7]:
loaded = np.load("training_data.npz", allow_pickle=True)


unprocessed_features = loaded["features"].tolist()
unprocessed_labels = loaded["labels"].tolist()

print(f'There are {len(unprocessed_features)} data points')
print(f'There were T = {len(unprocessed_features[0])} measurements performed for each data point')
print(f'The measurements were performed on {len(unprocessed_features[0][0])} qubits')
print(f'Example: 2nd experiment result of 8th data point -> {unprocessed_features[7][1]}')
print(f'Example: label for the 8th data point -> {unprocessed_labels[7]}')

There are 20 data points
There were T = 500 measurements performed for each data point
The measurements were performed on 51 qubits
Example: 2nd experiment result of 8th data point -> ['r', '-', 'i', 'i', 'r', '+', 'r', 'g', '+', '-i', 'r', '-', '-', 'g', '-i', '-', 'r', 'g', 'i', 'i', 'i', '-', 'r', '-i', 'r', 'i', 'r', '+', 'i', 'g', '-', '-i', '-', 'g', '-i', 'i', 'r', '-', '-', '-', 'i', 'i', '-i', 'g', '-i', 'g', 'r', '-', 'r', '+', '-']
Example: label for the 8th data point -> Z2


In the above,
- $\ket{g}$ is the atomic ground state, which is a $+1$-eigenstate of Pauli $Z$
- $\ket{r}$ is the highly excited Rydberg state, which is a $-1$-eigenstate of Pauli $Z$
- $\ket{+} = \frac{1}{\sqrt2}(\ket{g} + \ket{r})$, a $+1$-eigenstate of Pauli $X$
- $\ket{-} = \frac{1}{\sqrt2}(\ket{g} - \ket{r})$, a $-1$-eigenstate of Pauli $X$
- $\ket{+i} = \frac{1}{\sqrt2}(\ket{g} +i\ket{r})$, a $+1$-eigenstate of Pauli $Y$
- $\ket{-i} = \frac{1}{\sqrt2}(\ket{g} -i \ket{r})$, a $-1$-eigenstate of Pauli $Y$.

It is up to you how to convert the features into classical shadows and labels into numbers and then both into training data for your model. For example, you could assign $-1$ to $Z2$ and $+1$ to $Z3$.

**Note:** If you decide to define any helper classes/functions in a separate Python file, please submit it alongside your solution notebook, so we can run and grade it properly

In [8]:
# ─── Cell 1: Load & Classical-Shadow → PCA → 20 Features ───
import numpy as np
from sklearn.decomposition import PCA

# 1) Load raw measurement data
data = np.load("training_data.npz", allow_pickle=True)
raw_feats  = data["features"].tolist()   # (20 samples × 500 shots × 51 outcomes)
raw_labels = data["labels"].tolist()     # ("Z2"/"Z3") × 20

N, T, Q0 = len(raw_feats), len(raw_feats[0]), len(raw_feats[0][0])

# 2) Build classical shadows σ = 3|v⟩⟨v| – I
I2 = np.eye(2, dtype=complex)
basis = {
  'g': np.array([1,0],complex),
  'r': np.array([0,1],complex),
  '+': (1/np.sqrt(2))*np.array([1,1],complex),
  '-': (1/np.sqrt(2))*np.array([1,-1],complex),
  '+i':(1/np.sqrt(2))*np.array([1,1j],complex),
  '-i':(1/np.sqrt(2))*np.array([1,-1j],complex),
}
σ_op = {k:3*np.outer(v,v.conj()) - I2 for k,v in basis.items()}
norm = lambda s: '+i' if s.strip().replace('−','-')=='i' else s.strip().replace('−','-')

S = np.zeros((N, Q0, 2, 2), complex)
for i, sample in enumerate(raw_feats):
    acc = np.zeros((Q0,2,2), complex)
    for shot in sample:
        for q, o in enumerate(shot):
            acc[q] += σ_op[norm(o)]
    S[i] = acc / T

# 3) Convert to Bloch angles θₓ, θ_z
σx = np.array([[0,1],[1,0]], complex)
σz = np.array([[1,0],[0,-1]], complex)
rho = (S + I2) / 3
ex = np.real(np.trace(rho @ σx, axis1=2, axis2=3))
ez = np.real(np.trace(rho @ σz, axis1=2, axis2=3))
# map from [–1,1]→[0,π]
θx = (np.clip(ex, -1,1)+1)*(np.pi/2)
θz = (np.clip(ez, -1,1)+1)*(np.pi/2)

# 4) Flatten complex angles → shape (N, 204)
Xc = (θx + 1j*θz).reshape(N, -1)
Xreal = np.hstack([Xc.real, Xc.imag])   # shape (N, 408)

# 5) PCA → 20 components → normalize to [0,π]
pca = PCA(n_components=20, random_state=0)
Xp = pca.fit_transform(Xreal)           # (20,20)
mn, mx = Xp.min(0), Xp.max(0)
angles = (Xp - mn) / np.where(mx-mn==0,1,(mx-mn)) * np.pi

# 6) Final feature matrix & labels
X_feat = angles                           # (20,20)
y = np.array([ -1 if lbl=='Z2' else +1 for lbl in raw_labels])

print("Features:", X_feat.shape, "Labels:", y.shape)


Features: (20, 20) Labels: (20,)


# Defining a Quantum Model

In this section, you will create a QML model for classifying the quantum phases. This will include 3 stages:
- First, you will need to decide on the data encoding scheme, i.e. loading numerical features you obtained above into the quantum circuit.
- Then, you will need to come up with an ansatz - a parametrized quantum circuit, which will be optimized to perform classification.
- Finally, to readout classical information from the quantum model, you will need to perform some sort of measurement on the resultant quantum state. Perhaps, you could extract an expectation value of some Pauli-string $P \in \{I, X, Y, Z\}^{\otimes N}$, so that $\langle P \rangle < b$ is interpreted as $Z2$ and  $\langle P \rangle > b$ is interpreted as $Z3$ for some decision boundary $b$.

There are several approaches to QML in Classiq, linked below.

You may find the following guides useful:
- QML with Classiq: http://docs.classiq.io/latest/user-guide/read/qml_with_classiq_guide/
- Variational Model Example: https://github.com/Classiq/classiq-library/blob/main/algorithms/qaoa/maxcut/qaoa_max_cut.ipynb
- Hybrid QNN: https://docs.classiq.io/latest/explore/algorithms/qml/hybrid_qnn/hybrid_qnn_for_subset_majority/

Although the 2nd guide describes a hybrid model, **you may not implement a hybrid model**, the guide should only be used as a reference as to how to implement QML.

**Warning**: Training using the Classiq PyTorch integration may take a prohibitive amount of time. Consider this when choosing an approach.

In [11]:
from classiq import *
from classiq.execution import *
import numpy as np
from classiq.synthesis import synthesize, show

In [9]:
# ─── Cell 2: 5-qubit × 4-layer Circuit Definition & Synthesis ───
from classiq import (
    qfunc, CArray, CReal, QArray, QBit, Output,
    allocate, RX, RZ, RY, CZ,
    create_model, Constraints
)
from classiq.execution import (
    ExecutionPreferences, ClassiqBackendPreferences, ClassiqSimulatorBackendNames
)
from classiq.synthesis import synthesize, show

# 1) Hyperparameters
N_QUBITS  = 5
N_LAYERS  = 4
F_LEN     = N_QUBITS * N_LAYERS        # 20
N_WEIGHTS = N_QUBITS * N_LAYERS * 2    # RX+RZ per qubit per layer

# 2) Layered encoding (RX+RZ per layer & qubit)
@qfunc
def encode(f: CArray[CReal, F_LEN], wires: QArray) -> None:
    for layer in range(N_LAYERS):
        base = layer * N_QUBITS
        for q in range(N_QUBITS):
            RX(theta=f[base + q],       target=wires[q])
            RZ(theta=f[base + q],       target=wires[q])

# 3) Hardware‐efficient ansatz
@qfunc
def ansatz(w: CArray[CReal, N_WEIGHTS], wires: QArray) -> None:
    ptr = 0
    for _ in range(N_LAYERS):
        for q in range(N_QUBITS):
            RY(theta=w[ptr],    target=wires[q]); ptr += 1
            RZ(theta=w[ptr],    target=wires[q]); ptr += 1
        # CZ ring
        for q in range(N_QUBITS):
            CZ(ctrl=wires[q], target=wires[(q+1)%N_QUBITS])

# 4) Entry qfunc
@qfunc
def main(f: CArray[CReal, F_LEN],
         w: CArray[CReal, N_WEIGHTS],
         out: Output[QArray[QBit]]) -> None:
    allocate(N_QUBITS, out)
    encode(f, out)
    ansatz(w, out)

# 5) Synthesize
NUM_SHOTS = 1000
BACKENDS  = ClassiqBackendPreferences(
    backend_name=ClassiqSimulatorBackendNames.SIMULATOR
)

QMOD  = create_model(
    main,
    execution_preferences=ExecutionPreferences(
        num_shots=NUM_SHOTS,
        backend_preferences=BACKENDS
    ),
    constraints=Constraints(optimization_parameter="no_opt")
)
QPROG = synthesize(QMOD)
show(QPROG)

# width = 5, depth ≈ (2*5 + 5)*4 = 60 ops


Quantum program link: https://platform.classiq.io/circuit/2xEqlmlPHfB4DbU8cxR37O3OBsp?login=True&version=0.79.1


### Synthesis

Before training, you must synthesize your model into a quantum program. Placeholders for your parameters will be automatically generated.

You may find the following documentation useful: https://docs.classiq.io/latest/sdk-reference/synthesis/

In [60]:
# Synthesis
NUM_SHOTS = 1000
BACKENDS  = ClassiqBackendPreferences(
    backend_name=ClassiqSimulatorBackendNames.SIMULATOR
)
QMOD  = create_model(
    main,
    execution_preferences=ExecutionPreferences(
        num_shots=NUM_SHOTS, backend_preferences=BACKENDS
    ),
    constraints=Constraints(optimization_parameter="no_opt")
)
QPROG = synthesize(QMOD)
show(QPROG)

# Now width=8, depth≈(2+8)×1=10


Quantum program link: https://platform.classiq.io/circuit/2xELRD1kTCfLhhVgXfOSV8daFgm?login=True&version=0.79.1


# Training the Model

Here, you will optimize the weights in ansatz, so that the model can distiguish between the phases.

You can find the following Classiq tutorial and documentation useful:
- Execution: https://docs.classiq.io/latest/sdk-reference/execution/
- Execution Session: https://docs.classiq.io/latest/user-guide/execution/ExecutionSession/
- Executing With Parameters: https://docs.classiq.io/latest/qmod-reference/language-reference/quantum-entry-point/

It is highly recommended to use an ExecutionSession if you are executing the same circuit with different parameters many times. It is not needed to train parameters using the Classiq PyTorch integration.

If you are not using the PyTorch integration, you will need an objective (also known as a 'loss', or 'cost') function. Depending on your implementation, you will need to either minimize or maximize it in training.

In [ ]:
# ─── Cell 3: SPSA Training (64→256-shot) ───
import numpy as np
import matplotlib.pyplot as plt
from classiq.execution import ExecutionSession, ExecutionPreferences

# 1) Session factory
def make_sess(shots:int, fast:bool) -> ExecutionSession:
    prefs = ExecutionPreferences(
        num_shots=shots,
        backend_preferences=BACKENDS,
        fast=fast
    )
    return ExecutionSession(quantum_program=QPROG,
                            execution_preferences=prefs)

sess64  = make_sess(64,  fast=True)
sess256 = make_sess(256, fast=False)

# 2) Binder & avg-Z over all 5 qubits
def bind(tag, arr):
    return {f"{tag}_param_{i}": float(arr[i]) for i in range(len(arr))}

def avg_z(cnt):
    tot = sum(cnt.values())
    return sum(
        c * sum(+1 if b=='0' else -1 for b in bits)
        for bits,c in cnt.items()
    ) / (tot * N_QUBITS)

# 3) Batched forward
def batch_forward(sess, thetas, Xb):
    params = [
        { **bind("w", th), **bind("f", x) }
        for th,x in zip(thetas, Xb)
    ]
    res = sess.batch_sample(parameters=params)
    cnt = [r.counts if isinstance(r.counts,dict) else r.counts[0] for r in res]
    return np.array([avg_z(c) for c in cnt])

# 4) Loss & accuracy
num_weights = N_WEIGHTS
num_samples = X_feat.shape[0]

def hinge(sess, w):
    logits = batch_forward(sess, np.tile(w,(num_samples,1)), X_feat)
    return np.mean(np.maximum(0, 1 - y * logits))

def accuracy(sess, w):
    logits = batch_forward(sess, np.tile(w,(num_samples,1)), X_feat)
    return np.mean(np.sign(logits) == y)

# 5) SPSA hyperparameters
alpha, c0, A, gamma = 0.15, 0.10, 10.0, 0.1
MAX_IT = 300

theta    = 0.01 * np.random.randn(num_weights)
loss_log = []
acc_log  = []

# 6) SPSA loop
for k in range(MAX_IT):
    sess = sess64 if k < 50 else sess256
    ck   = c0 / (k+1)**gamma
    ak   = alpha         # no decay in code5
    delta= np.random.choice([+1,-1], num_weights)

    lp = hinge(sess, theta + ck*delta)
    lm = hinge(sess, theta - ck*delta)
    grad = (lp - lm) / (2 * ck * delta)
    theta -= ak * grad

    if k % 20 == 0 or k == MAX_IT-1:
        L = hinge(sess, theta)
        A = accuracy(sess, theta)
        loss_log.append(L)
        acc_log.append(A)
        stage = "64-shot" if k < 50 else "256-shot"
        print(f"iter {k:3d} | {stage:<8} | loss={L:.4f} | acc={A:.2%}")

# 7) Clean up & plot
sess64.close(); sess256.close()
plt.figure(figsize=(6,3)); plt.plot(loss_log,'o-'); plt.title("Hinge Loss")
plt.figure(figsize=(6,3)); plt.plot(acc_log, 's-'); plt.title("Accuracy")
plt.tight_layout(); plt.show()

# 8) Final circuit
show(QPROG)


iter   0 | 64-shot  | loss=1.1573 | acc=25.00%
iter  20 | 64-shot  | loss=1.1049 | acc=35.00%
iter  40 | 64-shot  | loss=1.0583 | acc=45.00%
iter  60 | 256-shot | loss=1.0151 | acc=50.00%
iter  80 | 256-shot | loss=0.9649 | acc=55.00%
iter 100 | 256-shot | loss=0.9279 | acc=55.00%
iter 120 | 256-shot | loss=0.8905 | acc=60.00%
iter 140 | 256-shot | loss=0.8516 | acc=65.00%
iter 160 | 256-shot | loss=0.8553 | acc=75.00%
iter 180 | 256-shot | loss=0.8269 | acc=75.00%
iter 200 | 256-shot | loss=0.7980 | acc=85.00%
iter 220 | 256-shot | loss=0.7736 | acc=85.00%
iter 240 | 256-shot | loss=0.7863 | acc=85.00%
iter 260 | 256-shot | loss=0.7777 | acc=85.00%


Training that takes too long may make it impossible to grade your submission.

# Testing the Model

Good job! Now it's time to see whether the model you designed can successfully perform the classification. For this, compare the predictions of your model to the actual labels.

If the model does not perform well, try modifying the encoding and/or the ansatz (by using different number of parameters/qubits/ansatz layers/...)

In [ ]:
### Your Code Goes Here: ###

## Grading

You will be evaluated on the accuracy, depth, width of your model and the number of parameters in your model.

The following function will return the width and depth of your model as they will be used in grading. Use it to self-evaluate your model.

In [67]:
from classiq import QuantumProgram

def get_metrics(qprog):
    """
    Extract circuit metrics from a quantum program.

    Parameters:
        qprog: The quantum program object.

    Returns:
        dict: A dictionary containing the circuit metrics:
              - "depth": Circuit depth
              - "width": Circuit width
    """
    circuit = QuantumProgram.from_qprog(qprog)

    metrics = {
        "depth": circuit.transpiled_circuit.depth,
        "width": circuit.data.width,
    }

    return metrics

In [68]:
print(get_metrics(QPROG))

{'depth': 27, 'width': 20}


# Submission

You will submit this notebook, your trained parameters, and your quantum model.

In [ ]:
# Do not change this cell

import os

def save_qprog(qprog, team_name: str, folder="."):
    assert isinstance(team_name, str)
    file_name = f"{team_name.replace(' ','_')}.qprog"
    with open(os.path.join(folder, file_name), 'w') as f:
        f.write(qprog.model_dump_json(indent=4))

def save_params(params, team_name: str, folder="."):
    assert isinstance(team_name, str)
    file_name = f"{team_name.replace(' ','_')}.npz"
    with open(os.path.join(folder, file_name), 'wb') as f:
        np.savez(f, params=params)

In [ ]:
# Change to your team name!!
TEAM_NAME = ""

# Insert your trained parameters here!
TRAINED_PARAMS = []

save_qprog(QPROG, team_name=TEAM_NAME)
save_params(params=TRAINED_PARAMS, team_name=TEAM_NAME)